In [28]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW
from xgboost import XGBClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle as pkl
import scipy
import os

from torch.nn import Linear, ReLU, Dropout
from torch.nn.functional import relu
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import gurobipy as gb

In [2]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = (
        False  # Force cuDNN to use a consistent convolution algorithm
    )
    torch.backends.cudnn.deterministic = (
        True  # Force cuDNN to use deterministic algorithms if available
    )
    torch.use_deterministic_algorithms(
        True
    )  # Force torch to use deterministic algorithms if available


In [3]:
try:
    corlat_dataset = pkl.load(open("Data/corlat/corlat_preprocessed.pickle", "rb"))
except:
    # move dir to /ibm/gpfs/home/yjin0055/Project/DayAheadForecast
    os.chdir("/ibm/gpfs/home/yjin0055/Project/DayAheadForecast")
    corlat_dataset = pkl.load(open("Data/corlat/corlat_preprocessed.pickle", "rb"))

In [4]:
# for each solution convert the dictionary to a list of values
solutions = [
    list(corlat_dataset[i]["solution"].values())
    for i in range(len(corlat_dataset))
]

In [5]:
# convert solutions_list to numpy array
solutions = np.array(solutions)

In [6]:
model_files = os.listdir("instances/mip/data/COR-LAT")

In [7]:
# get the indices of the binary variables
indices = []
for i in range(len(corlat_dataset)):
    indices.append(list(corlat_dataset[i]["solution"].keys()))

In [8]:
# convert indices to numpy array
indices = np.array(indices)

In [20]:
# read X_train, X_test, y_train, y_test from Data/corlat/ using numpy.load
X_train = np.load("Data/corlat/X_train.npy")
X_test = np.load("Data/corlat/X_test.npy")
y_train = np.load("Data/corlat/y_train.npy")
y_test = np.load("Data/corlat/y_test.npy")

In [25]:
# train and test indices
train_indices = np.load("Data/corlat/train_idx.npy")
test_indices = np.load("Data/corlat/test_idx.npy")

In [21]:
# load the xgboost model
with open("Models/Tabular/xgboost_model_corlat.pkl", "rb") as f:
    xgb_model = pkl.load(f)

In [22]:
y_pred = xgb_model.predict(X_test)

In [23]:
y_test = y_test.astype(np.int)
print("F1 score: ", f1_score(y_test, y_pred, average="micro"))
print("Precision: ", precision_score(y_test, y_pred, average="micro"))
print("Recall: ", recall_score(y_test, y_pred, average="micro"))

F1 score:  0.9237003241685676
Precision:  0.9232569302772111
Recall:  0.9241441441441441


/tmp/ipykernel_16531/3270043010.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test = y_test.astype(np.int)


In [19]:
# save xgboost model
xgb_model.save_model("Models/Tabular/xgboost_model_corlat.json")

# now test feasibility of the solutions

In [82]:
# basic optimization solving time
firstInstanceTest = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[0]])
firstInstanceTest.Params.Threads = 1
firstInstanceTest.optimize()

Read LP format model from file instances/mip/data/COR-LAT/cor-lat-2f+r-u-10-10-10-5-100-3.462.b208.000000.prune2.lp
Reading time = 0.01 seconds
obj: 470 rows, 466 columns, 1751 nonzeros
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen Threadripper 1920X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 1 threads

Optimize a model with 470 rows, 466 columns and 1751 nonzeros
Model fingerprint: 0x3db7ce21
Variable types: 366 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 17 rows and 9 columns
Presolve time: 0.00s
Presolved: 453 rows, 457 columns, 1715 nonzeros
Variable types: 360 continuous, 97 integer (97 binary)

Root relaxation: objective 4.122460e+02, 474 iterations, 0.01 seconds (0.00 wo

In [84]:
# we are going to use first instance of test data
firstInstanceTest = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[0]])
firstInstanceTest.Params.Threads = 1

# get indices of binary variables
firstInstanceTestBinaryIndices = indices[test_indices[0]]

# for this first instance of test data, we are going to use the xgboost prediction and fix the binary variables' values
# to the values predicted by xgboost

# get predictions from xgboost model
xgb_pred = xgb_model.predict(X_test[0].reshape(1, -1)).reshape(-1)

# get variables from the model
modelVars = firstInstanceTest.getVars()

# need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
for i in range(len(firstInstanceTestBinaryIndices)):
    modelVars[firstInstanceTestBinaryIndices[i]].setAttr("VType", "C")

    # for each index in firstInstanceTestBinaryIndices, set the value of the corresponding variable to the value predicted by xgboost
    modelVars[firstInstanceTestBinaryIndices[i]].setAttr("LB", xgb_pred[i])
    modelVars[firstInstanceTestBinaryIndices[i]].setAttr("UB", xgb_pred[i])
    
# After relaxing or fixing the binary variables, we can compute the IIS as before
firstInstanceTest.computeIIS()
# # Print the conflicting variables and constraints
# for v in firstInstanceTest.getVars():
#   if v.IISLB > 0 or v.IISUB > 0:
#     print(v.varName, "is part of the IIS")
# for c in firstInstanceTest.getConstrs():
#   if c.IISConstr > 0:
#     print(c.ConstrName, "is part of the IIS")

# only assign the predicted variables that are not in the IIS to warm start the model
for i, v in enumerate(firstInstanceTest.getVars()):
    if v.IISLB == 0 and v.IISUB == 0:
        if i in firstInstanceTestBinaryIndices:
            # print(v.varName, "is not part of the IIS")
            v.setAttr("VType", "B")
            v.setAttr("LB", 0)
            v.setAttr("UB", 1)
            v.setAttr("Start", xgb_pred[i])
            
    
    # else if the variable is in the IIS, 
    # get the relaxed variable and 
    # set the bounds to 0 and 1 for the relaxed binary variables
    else:
        if i in firstInstanceTestBinaryIndices:
            # print(v.varName, "is part of the IIS")
            v.setAttr("VType", "B")
            v.setAttr("LB", 0)
            v.setAttr("UB", 1)
            

firstInstanceTest.optimize()

Read LP format model from file instances/mip/data/COR-LAT/cor-lat-2f+r-u-10-10-10-5-100-3.462.b208.000000.prune2.lp
Reading time = 0.01 seconds
obj: 470 rows, 466 columns, 1751 nonzeros
Set parameter Threads to value 1
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9700000e+02   2.880000e+02   0.000000e+00      0s

IIS computed: 56 constraints and 73 bounds
IIS runtime: 0.01 seconds (0.00 work units)
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen Threadripper 1920X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 1 threads

Optimize a model with 470 rows, 466 columns and 1751 nonzeros
Model fingerprint: 0x499154c7
Variable types: 366 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]

User MIP start did not pr

In [77]:
# for each constraint, if constraint in IIS, get the slack value. Implement this.
for c in firstInstanceTest.getConstrs():
    if c.IISConstr > 0:
        print(c.ConstrName, "is part of the IIS")
        print(c.Slack)

c4 is part of the IIS
0.0
c112 is part of the IIS
0.0
c113 is part of the IIS
0.0
c116 is part of the IIS
0.0
c120 is part of the IIS
0.0
c122 is part of the IIS
0.0
c147 is part of the IIS
0.0
c162 is part of the IIS
0.0
c164 is part of the IIS
0.0
c174 is part of the IIS
-31.0
c175 is part of the IIS
0.0
c178 is part of the IIS
0.0
c182 is part of the IIS
0.0
c204 is part of the IIS
0.0
c208 is part of the IIS
0.0
c365 is part of the IIS
0.0
c366 is part of the IIS
0.0
c367 is part of the IIS
0.0
c368 is part of the IIS
0.0
c369 is part of the IIS
0.0
c370 is part of the IIS
0.0
c371 is part of the IIS
0.0
c372 is part of the IIS
0.0
c373 is part of the IIS
0.0
c374 is part of the IIS
0.0
c375 is part of the IIS
0.0
c376 is part of the IIS
0.0
c377 is part of the IIS
0.0
c378 is part of the IIS
0.0
c379 is part of the IIS
0.0
c380 is part of the IIS
0.0
c381 is part of the IIS
0.0
c382 is part of the IIS
0.0
c383 is part of the IIS
0.0
c384 is part of the IIS
0.0
c385 is part of the 

In [78]:
firstInstanceTest.getVars()[0]

<gurobi.Var x_1>

In [79]:
# only assign the predicted variables that are not in the IIS to warm start the model
for i, v in enumerate(firstInstanceTest.getVars()):
    if v.IISLB == 0 and v.IISUB == 0:
        if i in firstInstanceTestBinaryIndices:
            print(v.varName, "is not part of the IIS")
            v.setAttr("VType", "B")
            v.setAttr("LB", 0)
            v.setAttr("UB", 1)
            
    
    # else if the variable is in the IIS, 
    # get the relaxed variable and 
    # set the bounds to 0 and 1 for the relaxed binary variables
    else:
        if i in firstInstanceTestBinaryIndices:
            print(v.varName, "is part of the IIS")
            v.setAttr("VType", "B")
            v.setAttr("LB", 0)
            v.setAttr("UB", 1)

x_1 is not part of the IIS
x_2 is not part of the IIS
x_3 is not part of the IIS
x_4 is not part of the IIS
x_5 is not part of the IIS
x_6 is not part of the IIS
x_7 is not part of the IIS
x_8 is not part of the IIS
x_9 is not part of the IIS
x_10 is not part of the IIS
x_11 is not part of the IIS
x_12 is not part of the IIS
x_13 is not part of the IIS
x_14 is not part of the IIS
x_15 is not part of the IIS
x_16 is not part of the IIS
x_17 is not part of the IIS
x_18 is not part of the IIS
x_19 is not part of the IIS
x_20 is not part of the IIS
x_21 is not part of the IIS
x_22 is not part of the IIS
x_23 is not part of the IIS
x_24 is not part of the IIS
x_25 is not part of the IIS
x_26 is not part of the IIS
x_27 is not part of the IIS
x_28 is not part of the IIS
x_29 is not part of the IIS
x_30 is not part of the IIS
x_31 is not part of the IIS
x_32 is part of the IIS
x_33 is part of the IIS
x_34 is part of the IIS
x_35 is not part of the IIS
x_36 is not part of the IIS
x_37 is not p

In [80]:
# continue solving the model
firstInstanceTest.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen Threadripper 1920X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 470 rows, 466 columns and 1751 nonzeros
Model fingerprint: 0x3db7ce21
Variable types: 366 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 17 rows and 9 columns
Presolve time: 0.01s
Presolved: 453 rows, 457 columns, 1715 nonzeros
Variable types: 360 continuous, 97 integer (97 binary)

Root relaxation: objective 4.122460e+02, 474 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  412.24598    0   11          -  412.24598 

In [86]:
# the weights for each variable in the loss function should take the form of
# w_{ij} = exp(-c_i^T x^{i, j}) / sum(exp(-c_i^T x^{i, k})) for k = 1, ..., N_i
# where c_i is the vector of cost coefficient for training instance i, j is the index of the training instance, and N_i is the number of training instances


# we are going to use first instance of test data
firstInstanceTest = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[0]])

def custom_obj(model, y_true: np.ndarray, y_pred: np.ndarray):
    # Extract the cost coefficients for each training instance
    c = np.array([v.obj for v in model.getVars()])

    # Compute the IIS to find the list of violated constraints and variables
    model.computeIIS()

    # Get the list of violated constraints and variables
    violated_constrs = [c.ConstrName for c in model.getConstrs() if c.IISConstr]
    violated_vars = [v.VarName for v in model.getVars() if v.IISLB or v.IISUB]

    # Initialize the weights
    weights = np.zeros_like(y_true)

    # Compute the weights for each training instance
    for i in range(y_true.shape[0]):
        # Check if the assignment is feasible
        if not any([y_true[i, j] for j in violated_vars]):
            # Compute the numerator of the weight expression
            numerator = np.exp(-np.dot(c, y_true[i]))

            # Compute the denominator of the weight expression
            denominator = 0
            for k in range(y_true.shape[0]):
                if not any([y_true[k, j] for j in violated_vars]):
                    denominator += np.exp(-np.dot(c, y_true[k]))

            # Compute the weight for the current training instance
            weights[i] = numerator / denominator

    # Compute the gradient and hessian of the weighted binary logistic loss
    y_pred = 1.0 / (1.0 + np.exp(-y_pred))
    grad = weights * (y_pred - y_true)
    hess = weights * y_pred * (1.0 - y_pred)

    return grad, hess

# Define a wrapper function that takes only y_true and y_pred as arguments
def custom_obj_wrapper(y_true: np.ndarray, y_pred: np.ndarray):
    return custom_obj(firstInstanceTest, y_true, y_pred)

# Initialize an XGBClassifier model with the custom objective function
model = XGBClassifier(objective=custom_obj_wrapper, treemethod="hist")
    

In [88]:
y_train = y_train.astype(np.int)
model.fit(X_train, y_train)

[19:49:16] WARNING: /croot/xgboost-split_1675457761144/work/src/learner.cc:767: 
Parameters: { "treemethod" } are not used.

IIS computation: initial model status unknown, solving to determine model status
Presolve removed 17 rows and 9 columns
Presolve time: 0.00s
Presolved: 453 rows, 457 columns, 1715 nonzeros
Variable types: 360 continuous, 97 integer (97 binary)
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
IIS runtime: 0.02 seconds (0.00 work units)


/tmp/ipykernel_16531/484268324.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = y_train.astype(np.int)


GurobiError: Cannot compute IIS on a feasible model